In [0]:
# Replace with your actual storage account name and key
spark.conf.set(
    "fs.azure.account.key.60089061lab3.dfs.core.windows.net",
    "gaZRxrzOFqvnUGprNm3n8e0+b1wIZr3mG0bW8M+9f7IntlLoxv8LFs6LaF+3ssrv6DeMheqBnm5x+AStyIqsvw=="
)

In [0]:
# Load the books dataset from the silver layer
books = spark.read.parquet(
    "abfss://lakehouse@60089061lab3.dfs.core.windows.net/processed/books/"
)
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
    "abfss://lakehouse@60089061lab3.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()

In [0]:
from pyspark.sql.functions import col, length, trim, count, when
# Read raw (uncleaned) reviews from the silver layer
reviews = spark.read.parquet(
    "abfss://lakehouse@60089061lab3.dfs.core.windows.net/processed/reviews/"
)

# Peek at rows and schema
reviews.show(5, truncate=False)
reviews.printSchema()
# Basic profiling: counts and potential issues
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) ==
"") ).count()
print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id:{null_user_id}, NULL rating: {null_rating}")

In [0]:
print(f"Empty/NULL review_text: {empty_text}")

In [0]:
from pyspark.sql.functions import col, trim, length
# Start from the existing Parquet-loaded DataFrame
# (Assumes you already did: reviews = spark.read.parquet(".../processed/reviews/"))
df = reviews
# 1) Drop rows missing critical keys
df = df.filter(
col("review_id").isNotNull() &
col("book_id").isNotNull() &
col("user_id").isNotNull()
)
# 2) Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
col("rating_int").isNotNull() &
(col("rating_int") >= 1) &
(col("rating_int") <= 5)
)
# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
col("review_text").isNotNull() &
(length(col("review_text")) >= 10)
)
# 4) De-duplicate by review_id (keep arbitrary first; refine if you have timestamps)
df = df.dropDuplicates(["review_id"])


In [0]:
# 5) Select final shape
reviews_clean = df.select(
"review_id",
"book_id",
"user_id",
col("rating_int").alias("rating"),
"review_text"
)

In [0]:
# Write the cleaned reviews back to the silver layer (overwrite)
reviews_clean.write.mode("overwrite").parquet(
"abfss://lakehouse@60089061lab3.dfs.core.windows.net/processed/reviews/")
# Sanity check: re-read from disk and inspect schema and a few rows
reviews_verified = spark.read.parquet(
"abfss://lakehouse@60089061lab3.dfs.core.windows.net/processed/reviews/")
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

In [0]:
#--------------------- HOMEWORK I ----------------------

from pyspark.sql.functions import explode, col

books_with_authors = (
    books.select(
        "book_id",
        "title",
        "author_id",
        col("language_code").alias("language")
    )
    .join(
        authors.select("author_id", "name"),
        on="author_id",
        how="left"
    )
)

gold_table = books_with_authors.join(
    reviews_verified,
    on="book_id",
    how="left"
)

gold_columns = [
    "review_id",
    "book_id",
    "title",
    "author_id",
    "name",
    "user_id",
    "rating",
    "review_text",
    "language",
]

gold_table = gold_table.select([col(c) for c in gold_columns])

print("Gold Table Schema:")
gold_table.printSchema()

print("\nSample Rows (First 5):")
gold_table.show(5, truncate=False)


In [0]:
#--------------------- HOMEWORK I ----------------------

# --- 1. Write the curated gold_table to the gold zone ---
gold_path = f"abfss://lakehouse@60089061lab3.dfs.core.windows.net/gold/curated_reviews"

gold_table.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(gold_path)

# 2. Create an external location in Unity Catalog (run as a SQL command, only once)
# Replace <storage-credential-name> with your actual storage credential name
spark.sql("""
CREATE EXTERNAL LOCATION IF NOT EXISTS curated_gold_reviews_loc
URL 'abfss://lakehouse@60089061lab3.dfs.core.windows.net/gold'
WITH (STORAGE CREDENTIAL `60089061lab3`)
""")

# --- 3. Run a simple SQL query to confirm registration ---
print("Verification: First 5 rows from registered Delta table 'curated_reviews':")
spark.sql("""
SELECT 
    review_id, book_id, title, name AS author_name, rating
FROM curated_reviews 
LIMIT 5
""").show(truncate=False)

print(f"\nGold table successfully saved and registered at:")
print(f"   {gold_path}")
print(f"   SQL Table Name: curated_reviews")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4756610303449384>, line 14
      6 gold_table.write \
      7     .format("delta") \
      8     .mode("overwrite") \
      9     .option("overwriteSchema", "true") \
     10     .save(gold_path)
     12 # 2. Create an external location in Unity Catalog (run as a SQL command, only once)
     13 # Replace <storage-credential-name> with your actual storage credential name
---> 14 spark.sql("""
     15 CREATE EXTERNAL LOCATION IF NOT EXISTS curated_gold_reviews_loc
     16 URL 'abfss://lakehouse@60089061lab3.dfs.core.windows.net/gold'
     17 WITH (STORAGE CREDENTIAL `60089061lab3`)
     18 """)
     20 # --- 3. Run a simple SQL query to confirm registration ---
     21 print("Verification: First 5 rows from registered Delta table 'curated_reviews':")

File /databricks/spark/python/pyspark/sql/connect/session.py:821, in Spark